# General data questions and exploration


In [2]:
##TODO:
# Try ML with and without network metrics
# Test at different time periods
# Test other event datasets
# Check steps from book - change to best practice sklearn


# What correlates with witness label
# GPS accounts tend to be spam/businesses?
# Compare GPS from stream / profile / hand coding
# Explore age of account
# Is detecting co-occuring tags viable?
# What kind of data/user is likely to be deleted?
# Is user name change / user deletion/protection a useful predictor
# Compare Change in network, whether it's useful to collect.
# Check gps count, location in profile
# Check timezone distribution
# 'Ordinary person' vs bot/celeb/business/news -- using source field, tweet rate, timezone

# prop of gps -- users and tweets. Automated, instagram sourced?
# prop of sources
# prop of media/urls
# users with location on profile? Some set 'in solidarity'?
# Cycadian posting rythym - can identify real people vs bots?
# location via friend network?
# language

# \item Tweets which were automatically generated from Instagram posts were much more likely to include GPS coordinates, and as media, more likely to represent a ground truth. Therefore this content may be worth focusing on.
# \item Aid requests were very rare. Those that were identified were often reposts rather than originals, and are often referring to the same original message which begins to trend.
# \item Info for affected class should differentiate between immediate and non-immediate content. E.g. a call to mobilise a clean-up or rescue crew vs. a link to an insurance claim form.
# \item For `unrelated' messages, those which matched the keyword stream were highly represented by automated messages coming from a particular set of sources which presumably uses trending tags to gain exposure. This is easy to pre-filter.
# \item Geographically-tagged Tweets are predominantly either: Instagram cross-posts, or automatically generated job listings from a small set of sources (and therefore easy to pre-filter).
        

# Sum of network edge reciprocity
# k-cohesiveness -- Structural cohesion

In [3]:
### Initialisation ###
import os
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4]

EVENT_NAME = Event.objects.all()[0].name.replace(' ', '')
DIR = './data/harvey_user_location/'
DF_FILENAME = 'df_users.csv'

# Confirm correct database is set in Django settings.py
if 'Harvey' not in EVENT_NAME:
    raise Exception('Event name mismatch -- check database set in Django')

# Open original Dataframe
users_df = pd.read_csv(DIR + DF_FILENAME, index_col=0)
users_df.shape

(1500, 46)

First, given we are treating the data as binary-coded, we need to be aware of the third code of 'unsure'. In most comparisons, these will automatically be considered as the opposite code from what is being tested. We can instead manually set it to either value, or remove it entirely, given that it comprises a small proportion of the data.

In [4]:
unsure_code = (users_df.is_coded_as_witness == 0) & (users_df.is_coded_as_non_witness == 0)
print(sum(unsure_code), 'cases coded as \'unsure\'')

# Remove 'unsure' rows from data:
#users_df = users_df.loc[unsure_code==False]

# Assign 'unsure' rows to positive coded case:
#users_df.is_coded_as_witness = (users_df.is_coded_as_non_witness == False).astype(int)

# Assign 'unsure' rows to negative coded case:
#users_df.is_coded_as_non_witness = (users_df.is_coded_as_witness == False).astype(int)

31 cases coded as 'unsure'


## Geographic Metadata and Manual Coding
Manual coding of users targetted the perceived locality of the user to the event. We can compare the geographic metadata provided by Twitter to these codes to determine their usefulness as a predictor for this value.

### Profile Location Field
The first value to check is the location of a user as set in their profile. This is a user-set string. In an earlier notebook, this string was geocoded using Google Maps api and evaluated for whether it fell within the bounding box defined for this event. We can therefore check whether this test correlates with the coded value.

First, we check what proportion of users provide a value in the field. We can then generate a confusion matrix showing the agreement between the profile locality where provided, and the coded value.

In [5]:
us = User.objects.filter(user_class__gt=0)
#us = us.filter(coding_for_user__coding_id=1, coding_for_user__data_code__data_code_id__gt=0)
tot = us.count()
tot_loc = us.count() - (us.filter(location="") | us.filter(location__isnull=True)).count()

print('Total users: ', tot)
print('Total users with location filled: ', tot_loc)
print('Proportion: {:.4}%'.format((tot_loc/tot)*100))

print('\nProportion of coded users with location filled: {:.4}%'.format((sum(users_df.location.notna())/users_df.shape[0])*100))

print('\nProportion of coded users with parseable location filled: {:.4}%'.format(
    (users_df.loc[(users_df.is_non_local_profile_location + users_df.is_local_profile_location) > 0].shape[0]/users_df.shape[0])*100))


Total users:  31932
Total users with location filled:  25619
Proportion: 80.23%

Proportion of coded users with location filled: 79.4%

Proportion of coded users with parseable location filled: 77.47%


In [6]:
vals = users_df.loc[users_df["is_local_profile_location"] == 1]["is_coded_as_witness"].value_counts()
vals2 = users_df.loc[users_df["is_coded_as_witness"] == 1]["is_local_profile_location"].value_counts()

print('{} of {} ({:.4}%) users were classified as having a local profile'.format(sum(vals), len(users_df), sum(vals)/len(users_df)*100))
print('{} of {} ({:.4}%) users were coded as a witness'.format(sum(vals2), len(users_df), sum(vals2)/len(users_df)*100))
print('{} of {} ({:.4}%) users with local profile locations were coded as witness'.format(vals[1], sum(vals), vals[1]/sum(vals)*100))
print('{} of {} ({:.4}%) witness codes had a local profile'.format(vals2[1], sum(vals2), vals2[1]/sum(vals2)*100))

397 of 1500 (26.47%) users were classified as having a local profile
386 of 1500 (25.73%) users were coded as a witness
258 of 397 (64.99%) users with local profile locations were coded as witness
258 of 386 (66.84%) witness codes had a local profile


In [7]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )


def confusion_matrix_from_series(s1, s2):
    """
    Returns confusion matrix for two binary
    series
    """
    df = pd.concat([s1, s2], axis=1)
    return confusion_matrix(df, s1.name, s2.name)


def calc_agreement_coefs(df: pd.DataFrame):
    """
    Calculates Cohen's Kappa and
    Krippendorff's Alpha for a
    given confusion matrix.
    """
    arr = df.to_numpy()
    n = arr.sum()
    p_o = 0
    for i in range(len(arr)):
        p_o += arr[i][i]/n
    p_e = 0
    for i in range(len(arr)):
        p_e += (arr.sum(axis=1)[i] *
                arr.sum(axis=0)[i]) / (n*n)
    kappa = (p_o-p_e)/(1-p_e)
    
    coin_arr = np.transpose(arr) + arr
    exp_distribution = [sum(x) for x in coin_arr]
    p_e_krippendorf = sum([a * (a-1) for a in exp_distribution])/(2*n*((2*n)-1))
    alpha = (p_o - p_e_krippendorf) / (1-p_e_krippendorf)
    
    return p_o, kappa, alpha


def calc_agreement_metrics(df: pd.DataFrame):
    """
    Calculates various agreement metrics
    for a given binary confusion matrix.
    
    Assumes true condition as ROW heading and
    ascending integer labels.
    """
    arr = df.to_numpy()
    if len(arr) != 2:
        return null
    results = {}
    results['Prevalence'] = arr.sum(axis=0)[1]/arr.sum()
    results['Accuracy'] = (arr[0][0] + arr[1][1])/arr.sum()
    results['Prec'] = arr[1][1]/arr.sum(axis=1)[1]
    results['Recall'] = arr[1][1]/arr.sum(axis=0)[1]
    results['f1Score'] = (2 * results['Prec'] * results['Recall'])/(results['Prec']+results['Recall'])
    results['Specificity'] = arr[0][0]/arr.sum(axis=0)[0]
    results['FalseNegRate'] = arr[0][1]/arr.sum(axis=0)[1]
    p_o, kappa, alpha = calc_agreement_coefs(df)
    results['Cohen\'s Kappa'] = kappa
    results['Krippendorff\'s Alpha'] = alpha
    return results
    

In [8]:
# We exclude rows where either no profile location field was provided, 
# or the location was not parsed by the API:
loc_df = users_df.loc[(users_df.is_non_local_profile_location + users_df.is_local_profile_location) > 0]
print(users_df.shape[0] - loc_df.shape[0], 'rows with no parseable profile location value excluded')

conf = confusion_matrix(loc_df, 'is_local_profile_location', 'is_coded_as_witness')
conf

338 rows with no parseable profile location value excluded


is_coded_as_witness,0,1
is_local_profile_location,,
0,695,70
1,139,258


In [9]:
results = calc_agreement_metrics(conf)
res_df = pd.DataFrame.from_dict(results, orient='index', columns=['loc_prof_nona'])
res_df

,loc_prof_nona
Prevalence,0.282272
Accuracy,0.820138
Prec,0.649874
Recall,0.786585
f1Score,0.711724
Specificity,0.833333
FalseNegRate,0.213415
Cohen's Kappa,0.582731
Krippendorff's Alpha,0.581198


As excluding the ~20% of values with no parseable location field provided is not an option in practice, we must decide to either discard them (i.e. by default classify as non-local) or include them (default classify as local). The first option will inevitable discard true positive cases, thus reducing recall, whereas the latter will introduce false positives, reducing precision:

In [10]:
conf = confusion_matrix(users_df, 'is_local_profile_location', 'is_coded_as_witness')
print('loc_prof_notna')
print(conf)
res_df['loc_prof_notna'] = calc_agreement_metrics(conf).values()


conf = confusion_matrix_from_series(
    pd.Series(
                (users_df.is_local_profile_location) | (users_df.location.isna()).astype(int), 
                name='is_local_profile_location_or_na'
             ),
    users_df.is_coded_as_witness)
print('\nloc_prof_orna')
print(conf)
res_df['loc_prof_orna'] = calc_agreement_metrics(conf).values()


res_df

loc_prof_notna
is_coded_as_witness          0    1
is_local_profile_location          
0                          975  128
1                          139  258

loc_prof_orna
is_coded_as_witness                0    1
is_local_profile_location_or_na          
0                                714   80
1                                400  306


,loc_prof_nona,loc_prof_notna,loc_prof_orna
Prevalence,0.282272,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000
Prec,0.649874,0.649874,0.433428
Recall,0.786585,0.668394,0.792746
f1Score,0.711724,0.659004,0.560440
Specificity,0.833333,0.875224,0.640934
FalseNegRate,0.213415,0.331606,0.207254
Cohen's Kappa,0.582731,0.538603,0.341243
Krippendorff's Alpha,0.581198,0.538725,0.309098


The results above are as expected. Excluding empty fields gives a precision/recall of 0.650/0.668 whereas exluding them gives 0.433/0.793. The exclusion strategy provides the highest f1 score, however given the purpose of the algorithm must be considered when choosing how to weight precision and recall metrics. For example, given the algorithm is designed to curate the feed for human consumption, a high precision is only necessary if the rate of positive cases exceeds the humans' ability to parse the incoming stream. Where the rate is low, sacrificing precision is acceptable to present the human user with more cases which they can then manually filter. 

This concept will be explored in more depth later in the project. For now, it is sufficient to note the values as a baseline model.

### Tweet Stream Coordinates
When posting a Tweet, a user may attach geographic coordinates. The location of the device is provided by the hardware and automatically included with the Tweet (thus the user does not influence the input). A Tweet may also include, instead of specific coordinates, a 'Place' object -- a geographic region (defined by Twitter) which typically describes a location such as a city, state or other similarly-sized region.

To geolocate a user, we can therefore investigate their Twitter feed for any Tweets containing this geographic data and compare these to the bounding box of the observed event. The derived field therefore represents whether *any* of a user's Tweets were identified as 'local' during the event.

For this dataset, the Twitter feed for each observed user spanning the duration of the collection period was collected at the end of the collection period. The feed is therefore made up of Tweets detected during the collection period, and any other Tweets the user made during the period, before or after the detected Tweet, provided they existed at the end of the collection period.

Further work on this area should consider the following:
* Where a local Tweet has been detected, check the proportion of other Tweets containing geographic data.
* Where other geo-Tweets exist, check whether they are from the same point or move around -- consider recoding for where a user Tweets from within *and* without the bounding box.
* Where all geo-Tweets come from the same point, it is likely that location has been manually set (i.e. it is a storefront/business account). This may be verifiable by checking the Tweet source.

In [11]:
print('{} of {} ({:.3}%) users have Tweet from locality'.format(
    sum(users_df.has_tweet_from_locality), users_df.shape[0], 100*sum(users_df.has_tweet_from_locality)/users_df.shape[0]))

conf = confusion_matrix(users_df, 'has_tweet_from_locality', 'is_coded_as_witness')
conf

467 of 1500 (31.1%) users have Tweet from locality


is_coded_as_witness,0,1
has_tweet_from_locality,,
0,898,135
1,216,251


In [12]:
res_df['local_tw'] = calc_agreement_metrics(conf).values()
res_df

,loc_prof_nona,loc_prof_notna,loc_prof_orna,local_tw
Prevalence,0.282272,0.257333,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000,0.766000
Prec,0.649874,0.649874,0.433428,0.537473
Recall,0.786585,0.668394,0.792746,0.650259
f1Score,0.711724,0.659004,0.560440,0.588511
Specificity,0.833333,0.875224,0.640934,0.806104
FalseNegRate,0.213415,0.331606,0.207254,0.349741
Cohen's Kappa,0.582731,0.538603,0.341243,0.427080
Krippendorff's Alpha,0.581198,0.538725,0.309098,0.425219


While the recall of this metric is close (slightly less than) `loc_prof_notna`, there is no increase in precision and thus it is an inferior metric to predict the true condition.

We can however check whether the metric is capturing a different proportion of local users, and therefore improve upon the existing measures through combination. Using an OR condition will increase recall at the cost of precision; using an AND condition will increase precision at the cost of recall.

In [13]:
conf = confusion_matrix_from_series(
    pd.Series(
        ((users_df.is_local_profile_location) | (users_df.has_tweet_from_locality)),
        name = 'is_local_profile_location_or_local_tw'
        ),
    users_df.is_coded_as_witness)
# conf

res_df['loc_prof_notna_or_loc_tw'] = calc_agreement_metrics(conf).values()

conf = confusion_matrix_from_series(
    pd.Series(
        ((users_df.is_local_profile_location) & (users_df.has_tweet_from_locality)),
        name = 'is_local_profile_location_and_local_tw'
        ),
    users_df.is_coded_as_witness)
# conf

res_df['loc_prof_notna_and_loc_tw'] = calc_agreement_metrics(conf).values()

res_df

,loc_prof_nona,loc_prof_notna,loc_prof_orna,local_tw,loc_prof_notna_or_loc_tw,loc_prof_notna_and_loc_tw
Prevalence,0.282272,0.257333,0.257333,0.257333,0.257333,0.257333
Accuracy,0.820138,0.822000,0.680000,0.766000,0.788000,0.800000
Prec,0.649874,0.649874,0.433428,0.537473,0.558219,0.653571
Recall,0.786585,0.668394,0.792746,0.650259,0.844560,0.474093
f1Score,0.711724,0.659004,0.560440,0.588511,0.672165,0.549550
Specificity,0.833333,0.875224,0.640934,0.806104,0.768402,0.912926
FalseNegRate,0.213415,0.331606,0.207254,0.349741,0.155440,0.525907
Cohen's Kappa,0.582731,0.538603,0.341243,0.427080,0.524972,0.425170
Krippendorff's Alpha,0.581198,0.538725,0.309098,0.425219,0.515676,0.421208


As we can see from the table above, the highest precision is observed from the classifier `loc_prof_notna_and_loc_tw` at 0.656 (though at a great cost to recall). It should be noted that selecting for cases which satisfy both conditions may inadvertently select for a particular (as-yet unidentified) sub-category of user type and exclude other categories of value. The highest fscore comes from `loc_prof_notna_or_loc_tw` at 0.672, which has a precision of 0.558.

While the low scores of these metrics show that they cannot provide meaningful proxies for the manually assigned code, as classifiers they provide a suitable baseline from which to measure more sophisticated models.

## Tweet Source
An important distinguishing metadatum of a Tweet is the 'source' field, which represents the platform from which the Tweet was published. When creating a third-party application which can interact with the Twitter API, a developer must provide a descriptor string which populates this field. Because many third-party applications are designed for specific use-cases, this field provides useful information which characterises the motivations for conditions under which the Tweet was created. For example, the source `TweetMyJOBS` refers to a recruitment platform and thus is attached to Tweets advertising job listings.

We can look at the list of most common source from within the entire dataset and compare this to the sources detected during the collection period. (Note that the complete dataset will still contain a selection bias and does not necessarily characterise regular Twitter use)

In [44]:
# View most common sources from entire dataset:
from django.db.models import Count
from streamcollect.models import Tweet

ts = Tweet.objects.all()
print('Total Tweets:', ts.count(), '\n')

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in counts[:20]:
    if x['count'] > 10:
        print('{:.1f}% {}: {}'.format((x['count']/ts.count()*100), x['source'], x['count']))

Total Tweets: 1727438 

31.0% Twitter for iPhone: 535558
20.2% Twitter for Android: 348743
19.9% Twitter Web Client: 342908
5.5% IFTTT: 95360
2.7% Twitter for iPad: 45830
2.5% Twitter Lite: 43808
2.4% Instagram: 42045
1.9% TweetDeck: 32273
1.7% Facebook: 28536
1.6% Hootsuite: 27105
1.1% Paper.li: 18490
1.1% Botize: 18260
0.6% TweetMyJOBS: 10867
0.4% SafeTweet by TweetMyJOBS: 7650
0.4% Buffer: 6819
0.4% Google: 6469
0.3% WordPress.com: 5461
0.3% SocialNewsDesk: 4472
0.2% Mobile Web (M2): 3407
0.2% dlvr.it: 2972


In [46]:
# Count proportion of Tweets from first-party applications
first_party_sources = [
    'Twitter for iPhone',
    'Twitter for Android',
    'Twitter Web Client',
    'Twitter for iPad',
    'Twitter Lite',
    'TweetDeck',
    'Twitter for Windows',
    'Twitter for Mac',
    'Twitter for Windows Phone',
    'Twitter for BlackBerry',
    'Twitter for Android Tablets',
    'Twitter MMS'
    ]
fp_count = 0
for x in counts:
    if any([y in x['source'] for y in first_party_sources ]):
        fp_count += x['count']
print('{:.1f}% of total from first party clients: {}'.format((fp_count/ts.count()*100), fp_count))

78.5% of total from first party clients: 1355569


In [58]:
ts = Tweet.objects.filter(data_source=1)
print('Total for data_source=1 (keyword stream):', ts.count(), '\n')

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in counts[:20]:
    if x['count'] > 10:
        print('{:.1f}% {}: {}'.format((x['count']/ts.count()*100), x['source'], x['count']))

Total for source = 1 (keyword stream): 31303 

28.6% Twitter for iPhone: 8959
25.5% Twitter Web Client: 7989
16.2% Twitter for Android: 5086
5.6% Paper.li: 1754
3.1% Hootsuite: 981
2.3% Instagram: 733
2.3% IFTTT: 720
2.2% Facebook: 682
2.1% TweetDeck: 658
2.1% Twitter for iPad: 652
1.9% Twitter Lite: 595
0.9% Buffer: 274
0.4% SocialNewsDesk: 138
0.4% Sprout Social: 126
0.3% Error-log: 105
0.3% Botize: 93
0.2% Periscope: 70
0.2% VoiceStorm: 69
0.2% Google: 62
0.2% Twitter for Windows: 62


In [21]:
ts = Tweet.objects.filter(data_source__gt=1, coordinates_lat__isnull=False)
print('Total for data_source=3 (geo stream)', ts.count(), '\n')

fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in counts[:20]:
    if x['count'] > 10:
        print('{:.1f}% {}: {}'.format((x['count']/ts.count()*100), x['source'], x['count']))

Total for data_source=3 (geo stream) 15569 

76.6% Instagram: 11922
6.6% TweetMyJOBS: 1020
6.0% SafeTweet by TweetMyJOBS: 928
3.9% BubbleLife: 602
1.7% Foursquare: 267
1.5% Untappd: 234
0.8% Twitter for Android: 129
0.7% Hootsuite: 104
0.6% Twitter for iPhone: 87
0.4% circlepix: 55
0.3% TownTweet: 52
0.2% iOS: 28
0.2% Twitter for Android Tablets: 27
0.2% Crowdfire - Go Big: 25
0.1% Squarespace: 17
0.1% Twitter for Windows Phone: 15
0.1% Tweetbot for iΟS: 12


In [61]:
fp_count = 0
for x in counts:
    if any([y in x['source'] for y in first_party_sources ]):
        fp_count += x['count']
print('{:.1f}% of geo stream from first party clients: {}'.format((fp_count/ts.count()*100), fp_count))

1.7% of geo stream from first party clients: 262


Of the entire dataset of 1,727,438 Tweets, those published by first-party Twitter clients comprised 78.5% (1,355,569). In contrast, of the subset of 15,630 Tweets collected based on their location within the event's bounding box, only 1.7% (262) were published from first-party apps. Tweets crossposted by Instagram comprised 76.6% of the geotagged Tweets. The high incidence of Instagram posts in the geographic stream therefore suggest that Instagram posts are much more likely than Tweets to include geographic data, which is preserved during the crossposting process. We can check the rate across the entire dataset excluding the geo stream:

In [65]:
ts = Tweet.objects.filter(source='Instagram', data_source__lt=3)
print('Total Instagram Tweets (excl. geo stream):', ts.count())
ts_geo = ts.filter(coordinates_lat__isnull=False)
print('Total geotagged Instagram Tweets: {}, {:.1f}%'.format(ts_geo.count(), ts_geo.count()/ts.count()*100 ))

Total Instagram Tweets (excl. geo stream): 30123
Total geotagged Instagram Tweets: 9818, 32.6%


In [7]:
ts = Tweet.objects.filter(source__contains='TweetMyJOBS')
print('Total TweetMyJOBS Tweets:', ts.count())
ts_geo = ts.filter(coordinates_lat__isnull=False)
print('Total geotagged TweetMyJOBS Tweets: {}, {:.1f}%'.format(ts_geo.count(), ts_geo.count()/ts.count()*100 ))

Total TweetMyJOBS Tweets: 18517
Total geotagged TweetMyJOBS Tweets: 17150, 92.6%


As Instagram posts are traditionally based upon the publication of a recently-taken photo, the high incidence of geotagging makes this class of message highly useful in supporting the development of situational awareness.

Tweets were inspected by category (see code below) to classify the general use-case of each source and eliminate those which were only used for automated/unrelated purposes:

In [8]:
# Investigate Tweets by source to characterise source content:
# (samples are not limited to data_source>0)

ts = Tweet.objects.filter(data_source__gte=1)
fieldname = 'source'
counts = ts.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')
source_list = [x['source'] for x in counts if x['count'] > 10]

for source in source_list:
    print('__________________\n{}:\n'.format(source))
    ts = Tweet.objects.filter(source=source)
    for t in ts.order_by('?')[:15]:
#     for t in ts.filter(data_source__gte=1).order_by('?')[:15]: # To only sample detected Tweets
        print(t.text.replace('\n', ''))

__________________
Instagram:

#Ethimology is the STUDY of the ORIGIN of words and concepts @ Human History Project https://t.co/ln3CIj9rs0
Currently in #HOUSTON https://t.co/Q8UkvCFI9l
That's much better! Now #grandma #jihanjamal can see me! #grandmasboy #grandson #ilovemygrandson… https://t.co/LBAnrVdwOS
Should have left sooner. #flipthezip @ Minneapolis, Minnesota https://t.co/Bt8vUSEJ41
Day 1. Morning of pre #hurricaneharvey lamdfall #houston #tx 5 mins before the rain ☔️ this… https://t.co/0VTL6kMis8
Standing inside my front door @ Port Acres, Port Arthur, Texas https://t.co/tPdJGf3r0D
A true sign of the apocalypse.  Darwin &amp; Amelia sitting together.… https://t.co/Vn7eeXxoNX
Harvey sigue azotando sin compasion y ha decidido estacionarse en Houston. Nosotros estamos… https://t.co/nsDpr1Dmmj
The @astros have acquired @justinverlander from the tigers in exchange for prospects. #mlb… https://t.co/aTjPGJNuYH
Houston Missing: Have you seen this person? Please forward information:#Re

JTIC adds themes to November #milling, #cereal event. #WorldGrain @AemicFR   https://t.co/dg8ro5cQA0 https://t.co/uk4KVLi7Wk
Do you know the bare essentials to have on hand before emergency strikes? #HurricaneHarvey #Floodhttps://t.co/BJGJ0rLMTQ
More states have passed #telehealth parity laws, but fine print holds back adoption. https://t.co/Pswwrr0PA0 #healthcare #telemedicine
We offer many different services at Arlington Neuro. Here is a list of the procedures we offer. #arlingtonneuro https://t.co/vWPXCTqJTp
The 4 laws of #Digital Transformation https://t.co/Ih6aRRghR4 via @ThingsExpo #Cloud #IoT #Data #Computing
RT @tveitdal: Each big #catastrophe like a hurricane increases U.S.'s #poverty by 1% https://t.co/AOBzP9xUcN https://t.co/CTEpaf6Xv6
RT @robertocvaldez1: #vigilantesdeltiempo en Garland Tx!! https://t.co/uZ5FnAOj30
Physician offices, hospitals still trying to prepare for #MACRA reimbursement changes https://t.co/ClbupYqoHf
#LABORDAY #WEEKEND #CHICAGO https://t.co/6HnGZC8Mz9

Are you in Baja? #Lidia is heading your way tomorrow... #BreakingNewsMx https://t.co/JtEShTuhZO
Yes, military fidget spinners exist, and they are awesome https://t.co/fjGIonMsyJ https://t.co/55SQn3sZ6G
Magistrado Maikel Moreno denunció red que usó su nombre para extorsionar (+TUITS) publicado en https://t.co/VNc8qZNALM https://t.co/Q7KnbyY3nr
We are continuing to track dangerous storms! https://t.co/42e936Npf6
#Fetraharina niega la llegada de 60 mil toneladas de trigo rusohttps://t.co/uPiYmBTrHE #1Sep https://t.co/zUcCypdfJz
As #Afgahnistan &amp; #NorthKorea unfold, #American #War doctrine should be on our minds. https://t.co/b577ji7lbA  #Trump #tcot #Congress
Reportan lanzamiento de un misil desde #Norcorea https://t.co/YmIRf00nJT
@NuclearAnthro @mgerrydoyle It's from Andersen Air Force Base, Guam August 17, 2016 https://t.co/vgqBvwTdtn
@GeoJo22 Well said! After all, everything has to happen someplace ;)    #worldgeochat
#Character #SocialIntelligence #OtherPeopleMatter https://t.co/F

I'm at Boneheadz Sports Pub in San Antonio, TX https://t.co/d7NHmXPxmK
Celebrating Christine's birthday with the gang 🎉🎉 (at @ContigoAustin in Austin, TX) https://t.co/Bsk9nFH2GQ
I'm at Buc-ee's in Baytown, TX https://t.co/5AuIOWDRCh
Got some school supplies (at @Walgreens in San Antonio, TX) https://t.co/xpa5RqriKR
I'm at Ballpark South in Austin, TX https://t.co/MZ3UDXPg4n
Been trying to do this for weeks lol (@ Cutthroat in Houston, TX) https://t.co/PDB4P1nktY
I'm at SA Country in San Antonio, TX https://t.co/Ddmvp9McZF
Picking up more necessities!!! (@ Spec's Wines, Spirits &amp; Finer Foods - @specs1962 in Houston, TX w/ @chandrawest) https://t.co/uwpY53fbek
I'm at Home in Englewood, NJ, NJ https://t.co/cRHsWe4jXy
I'm at Starbucks in Round Rock, TX https://t.co/UmIdBgtCb9
I'm at @Harrods in London, Greater London https://t.co/kYKPyFcXc8
I'm at Lone Star Cafe in San Antonio, TX https://t.co/m0RS9oNoGs
When in the south side, might as well eat here. (@ Magnolia Cafe South - @magnoli

RT @JackPosobiec: This Democrat is losing it https://t.co/3wda5KYdel
RT @BojorquezCBS: INCREDIBLE how fast the water is rising in La Grange, TX where the Colorado River is now flowing through parts of town @C…
RT @MattSitkowski: This image and the forecast of what is still to fall.... This is surreal. #HoustonFlood #Harvey https://t.co/zK9WojsMAH
@nateferg27 Did your power go out as you were watching game of thrones?
@azamsharp be safe!
RT @_AndrewFriedman: Not for nothing, @brianstelter, but @maggieNYT's mom knows a thing or two about the game too. Taught many a young AE h…
RT @DenbrotS: Russia May Have Changed The Presidential Election Results, But Officials Are Afraid To Tell The Truthhttps://t.co/HXXNsaWrcG
@davidcarner Aw, thanks ❤️
@PixieLaBrador Is it fixed yet?  -Gracie #IWouldPostOnYelp
@mitchellvii @Sethrogen I stand corrected,After pushing Rogen to say some’n abt #Harvey he RTs. Turns out he values their lives @ 15 cents https://t.co/1v7LgxKHG3
RT @_Rico21: I swear to god in

...

Forget the rosy picture about #entrepreneurship. It is nice to talk about it from books and talking about the Richard #Bransons and the War…
Share your thoughts. https://t.co/dhhSaBrTVf
https://t.co/wBiHVbnqgo
Get it, #Mayweather. #cryptocurrency #bitcoin #MayweatherICO https://t.co/Xlyu6Q4aMe
How to work on a side project: https://t.co/Y0xBtDf55q"Don't waste energy on things that don't concern you." That's a big one.
Coast Guard Rescues 32 People as Harvey Churns Texas Waters https://t.co/v9KCfpLWED
@TBM_Janitorial - Over 30 years experience, at 97% customer satisifaction.  HUB, Minority women owned business. BOMA…https://t.co/fXicQINfYv
Interior’s High-Profile Whistleblower Gets a Boost From OSC and Legal Community https://t.co/6d9lfHfNjM
The real life FarmVille: 6 startups digitizing the family farm https://t.co/LicB67DlIz
South Australia calls for “the next generation” of renewable energy and energy storage projects https://t.co/0vOUe8XB1M
@ifeyinwa_kanu nice one ☝️ Climate-KIC ECC

RT @deo_whizard: What makes the letter “J” very special is "JESUS was born in JUDEA which is in JERUSALEM. His Earthly father... https:/…
RT @YaoMing: My thoughts are with the people of Houston today. Wishing everyone wellness and safety. Stay safe and stay strong Houston.
RT @deonaija: Floyd mayweather just got himself a new celebrity girlfriend https://t.co/ordr6nUumt https://t.co/Em0sJgFMbQ
RT @deonaija: https://t.co/TditzxcaOA https://t.co/58RdEK3yyf
Guiding and developing young people is uniquely rewarding but  responsibility is total. #Educating has reinforced my team's work @ngagedltd
RT @deonaija: Rihanna shows off her new ‘THICK’ figure in a bathing suit | See Photos https://t.co/Iamc2NZ51q
RT @AlamedaNL: #TeBuscamos Angel Guadalupe Aguilera  Guajardo. Nuevo Leon  #NvoLeon https://t.co/mjCq4zhG9M
RT @PierreRichardPR: Plus je suis triste, moins j'ai envie de parler… et là, je suis abattu par la disparition de Mireille… https://t.co/yX…
RT @AlamedaNL: #TeBuscamos Guillermo Nolas

Open House For Bellevue Neighborhood Enhancement Projects https://t.co/PJnSEMnmpz https://t.co/mmxzPn6ACj
Lugar paradisíaco en Oaxaca https://t.co/36odyESd1K #entretenimiento #HierveelAgua #Oaxaca #Hazclik https://t.co/TYcT46deSy
Hillary Clinton To Speak In Seattle https://t.co/pvO3RyU1tq https://t.co/cZT6zm5k4t
memes: Not a magician but i got a couple twix up my sleeve https://t.co/gZ8sWcIMNI https://t.co/tdIPMqEH4I
Video: Aparece un socavón en la carretera Mascota-Las Palmas https://t.co/HfYrpAraSl #seguridad #videos #derrumbes #hoyanco #Hazclik https://t.co/AlB45BLBAM
Ulta's 21 Days of Beauty Sale: The Best Deals to Score https://t.co/mKYUtWSNgj #Beauty_Product_Reviews https://t.co/uk6wRaSLsC
Trump sends out #fundraising email in middle of Texas emergency because *bangs head on desk* https://t.co/SU8l4kZiu2 https://t.co/cXvfumRkKs
GoHeels Exclusive: First Game Week Arrives https://t.co/2dUXPx90Ot https://t.co/PCcqMi25s2
Carolina Opens 2017 Saturday With Cal https://t.co/XON2bPWol4 h

https://t.co/y8BV4PPmJD  #BelgianGP #TheVoiceKids #DVglpCH #Harvey #MayweatherMcGregor Geneva Zurich Larousse du XXe Germany #AIBAFa 99
https://t.co/y8BV4PPmJD  #MumbaiRains #NorthKorea Sheik Farhan #Indonesia Kim Jong Un #USOpen Japan GOLD #HurricaneHarvey Arsenal 93
https://t.co/I63Req40YP  #digitalkmu #USOpen Houston #Harvey #Iraq #NationalLeague Swiss Conseil Geneva #GameOfThrones 55
https://t.co/ID69mnnXWR  #digitalkmu Houston #Harvey #gameofthrones #NationalLeague Swiss #Merkel Geneva #Boxing #ESCCongress 4
https://t.co/y8BV4PPmJD  #VMAs #efa17 #Harvey #GameOfThrones #ALTSVM Wahl Politiker Stau PS Blog Wien 96
https://t.co/y8BV4PPmJD  #ShaheedNawabAkbarBugti #BelgianGP #DVCVPCH #Harvey #srfarena Bondo Zurich #BritishGP #Bergell #climatechan 96
https://t.co/zo6VYCGAcj  #BelgianGP #DVCVPCH #Harvey #srfarena #Unspunnen Bondo Belgium Zurich #srfGP #AIBAFamily 26
https://t.co/1xwH42iSRr  #Indonesia #WGAFF #USOpen #LionMums2 #Harvey S. Korean Arsenal Japan Taylor Swift Game of Thrones 

@eBay to Launch Questions &amp; Answers on UK Site in October https://t.co/jzVKtVlMsp #ecommerce #smallbiz #sme #ebayseller #deal #auction #deal
Buy a memorial bracelet from #Kissimmee firefighters to help fallen #police officers' families. #NewlinLaw #DoGood https://t.co/XhhGaQjj8t
What3Words, New Kind of Street Address? https://t.co/FKBekcGgkg #ecommerce #smallbiz #crossbordertrade #globaltrade #sme #smb #what3words
#Hurricane #Harvey Impacts #eCommerce #Logistics &amp; #Deliveries https://t.co/6UyrneCHQB #hurricaneharvey #smallbiz #smallbusiness #smb #soho
40 Quotes that Will Calm Your Mind When You’re at a Crossroads https://t.co/mwClUuPq6M
ARCore: The New Augmented Reality Experience For Android https://t.co/iobcV4Y7S1 #arcore #ar #augmentedreality #smallbiz #ecommerce #android
#DanNewlin and Partners represent South Carolina woman injured at Aquatica in #Orlando #NewlinLaw #orlandoattorney https://t.co/kiEf9rC7ZX https://t.co/99ZRRfui4Y
Sometimes, space is a good thing. #rabbit #

* Instagram -- Crossposting images
* Twitter for iPhone
* Twitter Web Client
* Twitter for Android
* Paper.li -- Automated, seems like spam from a single source
* Hootsuite -- Suite, but appears to be used by 'regular users'
* TweetMyJOBS -- Automated job listings
* SafeTweet by TweetMyJOBS -- Automated job listings
* IFTTT -- Appears to be commonly used for cross-posting from eg Instagram
* Facebook
* TweetDeck
* Twitter for iPad
* BubbleLife -- Suite, news
* Twitter Lite
* Buffer -- Suite, news
* Foursquare -- Crossposting check-in app, sometimes informative, but mostly "I'm at x" template
* Untappd -- Crossposting check-in app (for beer drinkers), rarely informative
* SocialNewsDesk -- Suite, news
* Sprout Social -- Suite, news
* Error-log -- Automated bot / spam
* Botize -- All in Spanish, appeared to be only news
* Tweetbot for iΟS -- third-party app, regular users
* Periscope -- Crossposting, live video feeds
* VoiceStorm -- Suite, CRM
* Twitter for Windows
* Google -- Crossposting, mostly from Youtube (user likes or posts a video)
* Hearsay Social -- Suite, news
* despa ringtones -- Automated bot / spam
* circlepix -- Automated retail listings
* TweetCaster for Android -- Third party app
* TownTweet -- Automated forecasts
* Twitter for Mac
* Mobile Web (M2) -- (mobile.twitter.com)
* HubSpot -- Suite, CRM
* Crowdfire - Go Big -- Suite, news
* iOS
* Sprinklr -- Suite, CRM
* LinkedIn -- News article links
* Echofon -- Third-party app, regular users
* SeeYourWeather.com Galveston -- Automated weather bot
* WordPress.com -- Automated blog reposting
* GaggleAMP -- Suite, recruitment
* Twitter for Windows Phone
* Twitter for Android Tablets
* Radian6 -Social Media Management -- Automated news
* Convey: Make it post for you -- Automated news
* Cloudhopper -- Appears to be a Suite, CRM
* dlvr.it -- Automated news
* Twitter for BlackBerry
* Squarespace -- Automated blog reposting
* SocialFlow -- Suite, news
* Donate a Photo -- Looks like some J&J sponsored app
* Twitterrific -- Third-party app, regular users
* a7aa7o -- Automated bot / spam
* CoSchedule -- Suite, CRM
* despacitoapp -- Automated bot / spam
* shortage bot -- Automated bot / spam
* Bambu by Sprout Social -- Suite, CRM
* SocialPilot.co -- Suite, CRM
* Weather Message -- Automated weather reports
* RadarScope -- Weather enthusiast app


Therefore a list of sources can be created which are not used by 'regular users':

In [23]:
unwanted_sources = [
    'Paper.li',
    'TweetMyJOBS',
    'SafeTweet by TweetMyJOBS',
    'BubbleLife',
    'Buffer',
#     'Foursquare', ###
    'Untappd', ###
    'SocialNewsDesk',
    'Sprout Social',
    'Error-log',
    'Botize',
    'VoiceStorm',
    'Google', ###
    'Hearsay Social',
    'despa ringtones',
    'circlepix',
    'TownTweet',
    'HubSpot',
    'Crowdfire - Go Big',
    'Sprinklr',
    'LinkedIn',
    'SeeYourWeather.com Galveston',
    'WordPress.com',
    'GaggleAMP',
    'Radian6 -Social Media Management',
    'Convey: Make it post for you',
    'Cloudhopper',
    'dlvr.it',
    'Squarespace',
    'SocialFlow',
    'Donate a Photo',
    'a7aa7o',
    'CoSchedule',
    'despacitoapp',
    'shortage bot',
    'Bambu by Sprout Social',
    'SocialPilot.co',
    'Weather Message',
    'RadarScope']

In [34]:
ts = Tweet.objects.filter(data_source__gt=1)
print('Total for data_source=3 (geo stream)', ts.count())
ts2 = ts.exclude(source__in=unwanted_sources)
print('Total from accepted sources', ts2.count())
print('Total removed: {} ({:.2f}%)\n'.format((ts.count() - ts2.count()), 
                                           ((ts.count() - ts2.count())/ts.count())*100))

fieldname = 'source'
counts = ts2.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in [c for c in counts if c['count'] > 10]:
    print('{:.1f}% {}: {}'.format((x['count']/ts2.count()*100), x['source'], x['count']))

Total for data_source=3 (geo stream) 15569
Total from accepted sources 12628
Total removed: 2941 (18.89%)

94.4% Instagram: 11922
2.1% Foursquare: 267
1.0% Twitter for Android: 129
0.8% Hootsuite: 104
0.7% Twitter for iPhone: 87
0.2% iOS: 28
0.2% Twitter for Android Tablets: 27
0.1% Twitter for Windows Phone: 15
0.1% Tweetbot for iΟS: 12


In [33]:
ts = Tweet.objects.filter(data_source=1)
print('Total for data_source=1 (keyword stream)', ts.count())
ts2 = ts.exclude(source__in=unwanted_sources)
print('Total from accepted sources', ts2.count())
print('Total removed: {} ({:.2f}%)\n'.format((ts.count() - ts2.count()), 
                                           ((ts.count() - ts2.count())/ts.count())*100))

fieldname = 'source'
counts = ts2.values(fieldname).order_by(fieldname).annotate(count=Count(fieldname)).order_by('-count')

for x in [c for c in counts if c['count'] > 10]:
    print('{:.1f}% {}: {}'.format((x['count']/ts2.count()*100), x['source'], x['count']))

Total for data_source=1 (keyword stream) 31303
Total from accepted sources 28081
Total removed: 3222 (10.29%)

31.9% Twitter for iPhone: 8959
28.4% Twitter Web Client: 7989
18.1% Twitter for Android: 5086
3.5% Hootsuite: 981
2.6% Instagram: 733
2.6% IFTTT: 720
2.4% Facebook: 682
2.3% TweetDeck: 658
2.3% Twitter for iPad: 652
2.1% Twitter Lite: 595
0.2% Periscope: 70
0.2% Twitter for Windows: 62
0.2% Tweetbot for iΟS: 59
0.2% Twitter for Mac: 51
0.2% TweetCaster for Android: 51
0.2% Mobile Web (M2): 50
0.1% Echofon: 40
0.1% Twitter for BlackBerry: 19
0.1% Twitter for Windows Phone: 16
0.1% iOS: 15
0.0% Twitterrific: 14
